In [1]:
from stim_backend import NoisyStimBackend

Here's a new backend. Note that this has three error probabilities, for single-qubit, two-qubit and measurement gates. Errors are depolarizing for the former two, and bit flip for the latter.

In [2]:
p = 1e-2
n = 8
backend = NoisyStimBackend(
    num_qubits=n,
    basis_gates = ["id", "h", "x", "y", "z", "s", "cx"],
    p1=0,
    p2=0,
    pm=p
)

Let's try it out on the following circuit.

In [3]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.x(0)
qc.cx(0,1)
qc.measure_all()

This should yield `11` with certainty. So the probability that it doesn't is the probabilty of error. What do we expect for this?

* p1=p, p2=0, pm=0: p/2 (since only half of single-qubit Paulis are bit flips) = 0.005 (for p=1%)
* p1=0, p2=p, pm=0: 3p/4 (since 3/4s of two-qubit Paulis are bit flips) = 0.075 (for p=1%)
* p1=0, p2=0, pm=p: 2*p (since there are two ways it can happen) = 0.02 (for p=1%)

In [4]:
shots = 1e5
probs = {string:count/shots for string, count in backend.run(qc,shots=shots).result().get_counts().items()}
print('Total error')
print(round(1-probs['11'],4))

Total error
0.0201


Try it out. It seems to work!

Now let's see how we do with more qubits.

In [5]:
n = 127
backend = NoisyStimBackend(
    num_qubits=n,
    basis_gates = ["id", "h", "x", "y", "z", "s", "cx"],
    p1=p,
    p2=p,
    pm=p
)

qc = QuantumCircuit(n)
qc.h(range(n))
qc.measure_all()

probs = {string:count/shots for string, count in backend.run(qc,shots=shots).result().get_counts().items()}

This only takes a few seconds!

Unfortunately it doesn't seem to work with MirrorRB. Maybe because this backend is made to use `backend.run()` whereas Qiskit uses the sampler.

In [ ]:
from qiskit_device_benchmarking.bench_code.mrb import MirrorRB

n = 8
backend = NoisyStimBackend(
    num_qubits=n,
    basis_gates = ["id", "h", "x", "y", "z", "s", "cx"],
    p1=p,
    p2=p,
    pm=p
)

# number of shots per circuit
shots = 10000

# lengths of different mirror circuits to run
lengths = [2]+[4,10,20,50,100]

# set up the experiment object
exp = MirrorRB(
    range(backend.num_qubits),
    lengths,
    backend=backend,
    two_qubit_gate_density=0.25,
    num_samples=20,
    )
exp.set_run_options(
    shots=shots
)

rb_data = exp.run()
rb_data.block_for_results()


Job data not added for errored job [Job ID: 7cd7750e-1c7f-451b-b6a6-0a0f4786a796]
Error message: 


ExperimentData(MirrorRB, aa8abc92-c1cd-472a-8d9b-4bf982346153, job_ids=['7cd7750e-1c7f-451b-b6a6-0a0f4786a796'], metadata=<4 items>)

Provider for ExperimentData object doesn't exist, resulting in a failed attempt to retrieve data from the server; no stored result data exists
/Users/woottonjames/My Drive/Repos/qiskit-device-benchmarking/.venv/lib/python3.12/site-packages/qiskit_experiments/framework/base_analysis.py:173: UserWarning: ExperimentData object data is empty.

  warnings.warn("ExperimentData object data is empty.\n")
Provider for ExperimentData object doesn't exist, resulting in a failed attempt to retrieve data from the server; no stored result data exists
Provider for ExperimentData object doesn't exist, resulting in a failed attempt to retrieve data from the server; no stored result data exists
Provider for ExperimentData object doesn't exist, resulting in a failed attempt to retrieve data from the server; no stored result data exists
Provider for ExperimentData object doesn't exist, resulting in a failed attempt to retrieve data from the server; no stored result data exists
Analysis callback failed [Ex

In [7]:
exp.analysis.set_options(analyzed_quantity='Effective Polarization')
analysis = exp.analysis.run(rb_data)
analysis.block_for_results()

ExperimentData(MirrorRB, aa8abc92-c1cd-472a-8d9b-4bf982346153, job_ids=['7cd7750e-1c7f-451b-b6a6-0a0f4786a796'], metadata=<4 items>)

In [8]:
analysis.figure(0)

ExperimentEntryNotFound: 'Figure index 0 out of range.'